## Reserve and configure resources on Chameleon

## Create lease and reserve resources


In [31]:
import json
import os
import chi
import chi.ssh
import chi.network
from datetime import datetime, timedelta
from dateutil import tz

In [32]:
#Config with your project and site
project = os.getenv("OS_PROJECT_NAME")
chi.use_site('CHI@UC')
chi.set('project_name', project)

Now using CHI@UC:
URL: https://chi.uc.chameleoncloud.org
Location: Argonne National Laboratory, Lemont, Illinois, USA
Support contact: help@chameleoncloud.org


In [33]:
# Name resources with your username for easier identification
username = os.getenv("USER")
prefix = username + "_k8s-ml_"
server_name = prefix + 'Server'
network_name = prefix + 'Net'
subnet_name = prefix + 'Subnet'
router_name = prefix + 'Router'
lease_name = prefix + 'Lease'

In [34]:
#Server attributes
image_name='CC-Ubuntu18.04'
node_type="compute_cascadelake_r"
server_count=3
physical_network="physnet1"
subnet_cidr = '192.168.1.0/24'

#### Create the Network

In [35]:
# Create a network out of provider network physnet1
network = chi.network.create_network(network_name, provider='physnet1')
network_id = network['id']

print(f'Network ID: {network_id}')

Network ID: b4cf74dc-30a1-486d-b3f9-88a8c61a5014


#### Configure the Network


In [36]:
subnet = chi.network.create_subnet(subnet_name, network_id, cidr=subnet_cidr, gateway_ip=None)
router = chi.network.create_router(router_name, gw_network_name='public')
chi.network.add_subnet_to_router(router['id'], subnet['id'])

{'id': 'a315c62f-b5da-4360-9e21-bae0c3ee80ad',
 'tenant_id': '49f47d2c64e64937840c3f7c663a37b2',
 'port_id': '2ab25402-bf49-47a7-bb92-0df69922753b',
 'network_id': 'b4cf74dc-30a1-486d-b3f9-88a8c61a5014',
 'subnet_id': 'ecf6a3eb-2c57-4dfc-adc3-de79cac5a0fe',
 'subnet_ids': ['ecf6a3eb-2c57-4dfc-adc3-de79cac5a0fe']}

#### Create a lease with network and 3 servers

In [37]:
import chi.lease

BLAZAR_TIME_FORMAT = '%Y-%m-%d %H:%M'

# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=3)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
chi.lease.add_node_reservation(reservation_list, count=server_count, node_type=node_type)
chi.lease.add_fip_reservation(reservation_list, count=server_count)

# Create the lease
lease = chi.lease.create_lease(lease_name, 
                               start_date=start_date,
                               end_date=end_date,
                               reservations=reservation_list)

lease_id = lease["id"]

chi.lease.wait_for_active(lease_id)

{'created_at': '2023-05-15 03:21:59',
 'updated_at': '2023-05-15 03:22:11',
 'id': '7c7ac145-e3eb-47b2-b4fe-22d31e9ebe4c',
 'name': 'cp3793_nyu_edu_k8s-ml_Lease',
 'user_id': '50de6f77f6d1941774ecf322c9f0ad2a3e2c128f4707c3278d98fd4a98b86d85',
 'project_id': '49f47d2c64e64937840c3f7c663a37b2',
 'start_date': '2023-05-15T03:22:00.000000',
 'end_date': '2023-05-18T03:21:00.000000',
 'trust_id': 'af4d59fbffce44ee89b273442e5b96c7',
 'status': 'ACTIVE',
 'degraded': False,
 'reservations': [{'created_at': '2023-05-15 03:22:00',
   'updated_at': '2023-05-15 03:22:09',
   'id': '3b32168e-d499-4bd4-a2a1-37a820312fec',
   'lease_id': '7c7ac145-e3eb-47b2-b4fe-22d31e9ebe4c',
   'resource_id': '817600be-1524-412b-9c58-2b8428802621',
   'resource_type': 'virtual:floatingip',
   'status': 'active',
   'missing_resources': False,
   'resources_changed': False,
   'network_id': '44b38c44-2a42-4b6d-b129-6c8f1b2a1375',
   'amount': 3,
   'required_floatingips': []},
  {'created_at': '2023-05-15 03:21:59'

#### Get the Reservations

In [38]:
compute_reservation_id = [reservation for reservation in lease['reservations'] if reservation['resource_type'] == 'physical:host'][0]['id']
floatingip_reservation_id = [reservation for reservation in lease['reservations'] if reservation['resource_type'] == 'virtual:floatingip'][0]['id']

print(f"compute_reservation_id: {compute_reservation_id}")
print(f"floatingip_reservation_id: {floatingip_reservation_id}")

compute_reservation_id: 5fa6cf52-8984-490f-8011-8470b3f2a543
floatingip_reservation_id: 3b32168e-d499-4bd4-a2a1-37a820312fec


#### Start the Server

In [39]:
#create the server
servers = []
for i in range(server_count):
    servers.append(chi.server.create_server(server_name+f"-{i}", 
                                  compute_reservation_id,
                                  network_id=network_id,
                                  nics=[{"net-id": network_id,"v4-fixed-ip": f"192.168.1.1{i}" }],
                                  image_name=image_name,
                                  count=1))


#### Associate the floating IPs

In [40]:
nodes = {}

for j,i in enumerate(servers):
    chi.server.wait_for_active(i.id)
    floating_ip = chi.server.associate_floating_ip(i.id)
    nodes[j] = floating_ip

    print(f'Floating IP of node_{j}: {floating_ip}')

Floating IP of node_0: 192.5.86.229
Floating IP of node_1: 192.5.86.227
Floating IP of node_2: 192.5.86.225


#### Wait for SSH access to all servers

In [42]:
for node_ip in nodes.values():
    chi.server.wait_for_tcp(node_ip, port=22)

### Configuring the servers

#### Install keys for SSH access between all servers

In [43]:
for node_ip in nodes.values():
    remote = chi.ssh.Remote(node_ip)
    remote.run('ssh-keygen -t rsa -b 4096 -f ~/.ssh/id_rsa -q -N ""')
    public_key = remote.run('cat ~/.ssh/id_rsa.pub').tail("stdout")[2:]
    public_key = public_key.replace("\n", "")
    for other_node_ip in nodes.values():
        remote = chi.ssh.Remote(other_node_ip)
        remote.run(f'echo {public_key} >> ~/.ssh/authorized_keys') 

/opt/conda/lib/python3.10/site-packages/paramiko/client.py:835: UserWarning: Unknown ssh-ed25519 host key for 192.5.86.229: b'da1208db4c3f73add8640801c38b9c4f'
  warnings.warn(


ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQC048V5PkFVZ7e302T32Hi/+mTCC6Kw9b6jNXV63vouA1GlvnCEtpP5qBOH24yLTfMDUPZnv6bCz2TGyA2436j76LVH79QxvAc/4XaVrfKpJCc147TrTjJbkM+jZbd0AFl64l0KsxLbAKzt+bafnRIT3pLdJsBU9Oap3LZC5HsdrOqlwdcozQ95awA2IGexbEBf4OCXY6psgsic4GvaocyQGoQCxp0ckniUuyr0QE1GFU31CM5G5GhE2HCi1ThZHjzTrfvnJcWRPsv/sz7uMffnBrl02yaRzItEiUdugGikK9Wif2iyCJ9HRD7cp0LpBFbDub/ZU4Y314+iyp2vG6241AdW3ohfit/CFBXgWTdPcyjoByc7HisFMveUcPaRdc4o8A8Y9xdu7ssm6om1QjBnAQnQiGxHBomB943dTXQhMrWy6LKiuBI69LadW37yJX2dHDNcFqqJOLHeonxgx/8fjjYCPy8Fe33YcpFuSfSng8ZHUD7k+p2S5NPWcvbMHAky2fwJiuhzJXBYqq2ofgKYm9W+EOut+Der3YFLdd+hzM1pWBWxR+c06a8Udqlp1Fq1H3mwqIBvI6EYq5wBbE4OmRCc7P0Q8O70HTrzNupWJia9cKBAycDzrnAgZ97XJjlb97n3jCX3WwzAiiHYLrngirxbHil9/8VRKO3iZKRLlw== cc@cp3793-nyu-edu-k8s-ml-server-0


/opt/conda/lib/python3.10/site-packages/paramiko/client.py:835: UserWarning: Unknown ssh-ed25519 host key for 192.5.86.227: b'8b6af5b724448cd8ebbf66224f10f495'
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/paramiko/client.py:835: UserWarning: Unknown ssh-ed25519 host key for 192.5.86.225: b'a8894a75a2f753e8b1bff2ee64ee7b04'
  warnings.warn(


ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQCpHOJ2tPffMAS5+ZlF/MfdV1fdO+gIyJ0ikb3NRkf7HMplD3BVtL8ddinrniWwaJNDvYNjHMFGeJ05lejjXzd+jp+XisdC2JvCbPpCZEZmav+zv4JHX0RgkcoCQjp+X1PwTGmYOiYJFf1hq2iOCZetbh9CbIMXk3yOdg2i9PLwLUsHk6ZNYiljfVmTJFyOz9bsK7uI+u/9hHTWQovDzRsrH1hG+O5GGqypMxAXO3tvcKHDhSbObw6rMK8w33T/kXHndx21hkgBsR3CKAx+HZJPYZNUT1/UpFmVV/eWgIcQuJ7/O9PhO5mWrWhYBR3uhfFYcocfZWKFtxrK6lgKf2iWFEmmbH67s0XArs54lPX6rclpT8ZsBscJipRYnqkKR1pfE7yKeMqduuuyGWGzznEH8WCgKSR3u5m6wkW2ajmoalMTBnd65uTHbV6zToeosRGjqvsRSmtFsFHVUAQUXuNaxLojcweUwodWAXE5jo3v0PQiifTVE6Lml4KUDU/cOCxIUeiMELAzhnahedKD9QqaY11BBuXIVu2ESXLY9ErF38F4kfcdIhtjjs4AtqwN50Mkx70uIDh2wuwCwG4pZKTifvtfddG23Uf6G/kRwNoSpYK0A36J/+Y/dpu6ZPcPiYA/d7jOvE9vM5zxRT93ttg3KQcXmQmBt7vBzLc/7aZNww== cc@cp3793-nyu-edu-k8s-ml-server-1
ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQC21tgygQn5Io8+oJfczYugHdNgpMyzSY1gdcqAhL60xfKSVV2oj65u3z2kfE0FEiVFTc5JDUuwUVVQtONINxIO/I1f+mkJic3bV+w533UKcL6nbNWWNApdxhj5ELgwWoRg0Dzatb3vsvcXFiFzEmTDlDJ1yX9ZYUXYdoJxUojS6oOPSlBSpPZJVu6leaY+lbDLzR6esYbKa

#### Store the IP address of each host

When creating the cluster, we will need these addresses, which are assigned by DHCP and so are different each time.

In [44]:
physical_ips=[]
for node_ip in nodes.values():
    chi.server.wait_for_tcp(node_ip, port=22)
    remote = chi.ssh.Remote(node_ip)
    physical_ips.append(remote.run('hostname -I').tail("stdout")[2:-1])

192.168.1.10 
192.168.1.11 
192.168.1.12 


#### Use Kubespray to prepare a Kubernetes cluster

In [45]:
ansible_node = nodes[0]
remote = chi.ssh.Remote(ansible_node)
remote.run("sudo apt install virtualenv")
remote.run("virtualenv -p python3 myenv")
remote.run("git clone https://github.com/kubernetes-sigs/kubespray.git")
remote.run("cd kubespray; git checkout release-2.16")
remote.run("source myenv/bin/activate; cd kubespray; pip3 install -r requirements.txt")
remote.run("cd kubespray; cp -rfp inventory/sample inventory/mycluster")
remote.run("rm -rf kubespray/contrib/inventory_builder/inventory.py")
remote.run("rm -rf kubespray/inventory/mycluster/group_vars/k8s_cluster/addons.yml")
remote.run("wget https://raw.githubusercontent.com/teaching-on-testbeds/k8s-ml/main/config/inventory.py -O kubespray/contrib/inventory_builder/inventory.py")
remote.run("wget https://raw.githubusercontent.com/teaching-on-testbeds/k8s-ml/main/config/addons.yml -O kubespray/inventory/mycluster/group_vars/k8s_cluster/addons.yml")
remote.run(f"source myenv/bin/activate; declare -a IPS=({physical_ips[0]} {physical_ips[1]} {physical_ips[2]});"+"cd kubespray; CONFIG_FILE=inventory/mycluster/hosts.yaml python3 contrib/inventory_builder/inventory.py ${IPS[@]}")

Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  virtualenv
0 upgraded, 1 newly installed, 0 to remove and 0 not upgraded.
Need to get 4476 B of archives.
After this operation, 30.7 kB of additional disk space will be used.
Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu bionic/universe amd64 virtualenv all 15.1.0+ds-1.1 [4476 B]


debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Fetched 4476 B in 0s (23.7 kB/s)
Selecting previously unselected package virtualenv.
(Reading database ... 63374 files and directories currently installed.)
Preparing to unpack .../virtualenv_15.1.0+ds-1.1_all.deb ...
Unpacking virtualenv (15.1.0+ds-1.1) ...
Setting up virtualenv (15.1.0+ds-1.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Already using interpreter /usr/bin/python3
Using base prefix '/usr'
New python executable in /home/cc/myenv/bin/python3
Also creating executable in /home/cc/myenv/bin/python
Installing setuptools, pkg_resources, pip, wheel...done.


Cloning into 'kubespray'...


Branch 'release-2.16' set up to track remote branch 'release-2.16' from 'origin'.


Switched to a new branch 'release-2.16'


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for ansible: filename=ansible-2.9.20-py3-none-any.whl size=16203449 sha256=379211ae4d49761a89c1e2e86140c68c2215e101f19d61e27c6c0f2ade6174f6
  Stored in directory: /home/cc/.cache/pip/wheels/0f/36/b7/907fa25c2228a59030698d2ac557ba5c694b648dede82fb7e1
Successfully built ansible


--2023-05-15 03:53:40--  https://raw.githubusercontent.com/teaching-on-testbeds/k8s-ml/main/config/inventory.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17940 (18K) [text/plain]
Saving to: ‘kubespray/contrib/inventory_builder/inventory.py’

     0K .......... .......                                    100%  154M=0s

2023-05-15 03:53:40 (154 MB/s) - ‘kubespray/contrib/inventory_builder/inventory.py’ saved [17940/17940]

--2023-05-15 03:53:40--  https://raw.githubusercontent.com/teaching-on-testbeds/k8s-ml/main/config/addons.yml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTT

DEBUG: Adding group all
DEBUG: Adding group kube_control_plane
DEBUG: Adding group kube_node
DEBUG: Adding group etcd
DEBUG: Adding group k8s_cluster
DEBUG: Adding group calico_rr
DEBUG: adding host node-0 to group all
DEBUG: adding host node-1 to group all
DEBUG: adding host node-2 to group all
DEBUG: adding host node-0 to group etcd
DEBUG: adding host node-1 to group etcd
DEBUG: adding host node-2 to group etcd
DEBUG: adding host node-0 to group kube_control_plane
DEBUG: adding host node-1 to group kube_control_plane
DEBUG: adding host node-0 to group kube_node
DEBUG: adding host node-1 to group kube_node
DEBUG: adding host node-2 to group kube_node


<Result cmd='source myenv/bin/activate; declare -a IPS=(192.168.1.10 192.168.1.11 192.168.1.12);cd kubespray; CONFIG_FILE=inventory/mycluster/hosts.yaml python3 contrib/inventory_builder/inventory.py ${IPS[@]}' exited=0>

In [46]:
# need a specific versino of pyOpenSSL
remote.run("source myenv/bin/activate; sudo rm -rf /usr/local/lib/python3.6/dist-packages/OpenSSL")
remote.run("source myenv/bin/activate; sudo rm -rf /usr/local/lib/python3.6/dist-packages/pyOpenSSL-22.1.0.dist-info/")
remote.run("source myenv/bin/activate; sudo pip3 install pyOpenSSL==22.0.0")

  Attempting uninstall: pyOpenSSL
    Found existing installation: pyOpenSSL 17.5.0
    Uninstalling pyOpenSSL-17.5.0:
      Successfully uninstalled pyOpenSSL-17.5.0


<Result cmd='source myenv/bin/activate; sudo pip3 install pyOpenSSL==22.0.0' exited=0>

In [47]:
remote.run("source myenv/bin/activate; cd kubespray; ansible-playbook -i inventory/mycluster/hosts.yaml  --become --become-user=root cluster.yml")


PLAY [localhost] ***************************************************************
Monday 15 May 2023  03:54:12 +0000 (0:00:00.048)       0:00:00.048 ************ 

TASK [Check 2.9.0 <= Ansible version < 2.11.0] *********************************
ok: [localhost] => {
    "changed": false,
    "msg": "All assertions passed"
}
Monday 15 May 2023  03:54:12 +0000 (0:00:00.014)       0:00:00.063 ************ 

TASK [Check that python netaddr is installed] **********************************
ok: [localhost] => {
    "changed": false,
    "msg": "All assertions passed"
}
Monday 15 May 2023  03:54:12 +0000 (0:00:00.013)       0:00:00.076 ************ 

TASK [Check that jinja is not too old (install via pip)] ***********************
ok: [localhost] => {
    "changed": false,
    "msg": "All assertions passed"
}


[WARNING]: Could not match supplied host pattern, ignoring: kube-master



PLAY [Add kube-master nodes to kube_control_plane] *****************************
skipping: no hosts matched

PLAY [Add kube-node nodes to kube_node] ****************************************
skipping: no hosts matched


[WARNING]: Could not match supplied host pattern, ignoring: kube-node
[WARNING]: Could not match supplied host pattern, ignoring: k8s-cluster



PLAY [Add k8s-cluster nodes to k8s_cluster] ************************************


...

[WARNING]: flush_handlers task does not support when conditional


Monday 15 May 2023  03:54:46 +0000 (0:00:00.032)       0:00:33.279 ************ 
Monday 15 May 2023  03:54:46 +0000 (0:00:00.032)       0:00:33.312 ************ 

TASK [container-engine/containerd-common : gather os specific variables] *******
ok: [node-0] => (item=/home/cc/kubespray/roles/container-engine/containerd-common/vars/../vars/ubuntu.yml)
ok: [node-1] => (item=/home/cc/kubespray/roles/container-engine/containerd-common/vars/../vars/ubuntu.yml)
ok: [node-2] => (item=/home/cc/kubespray/roles/container-engine/containerd-common/vars/../vars/ubuntu.yml)
Monday 15 May 2023  03:54:46 +0000 (0:00:00.055)       0:00:33.367 ************ 
Monday 15 May 2023  03:54:46 +0000 (0:00:00.046)       0:00:33.414 ************ 
Monday 15 May 2023  03:54:46 +0000 (0:00:00.033)       0:00:33.447 ************ 
Monday 15 May 2023  03:54:46 +0000 (0:00:00.032)       0:00:33.480 ************ 
Monday 15 May 2023  03:54:46 +0000 (0:00:00.034)       0:00:33.514 ************ 
Monday 15 May 2023  03:54:46 +

[WARNING]: noop task does not support when conditional



TASK [download : prep_kubeadm_images | Create kubeadm config] ******************
changed: [node-1]
changed: [node-0]
Monday 15 May 2023  03:55:20 +0000 (0:00:00.372)       0:01:07.356 ************ 

TASK [prep_kubeadm_images | Copy kubeadm binary from download dir to system path] ***
changed: [node-0]
changed: [node-1]
Monday 15 May 2023  03:55:20 +0000 (0:00:00.325)       0:01:07.682 ************ 

TASK [download : prep_kubeadm_images | Set kubeadm binary permissions] *********
ok: [node-0]
ok: [node-1]
Monday 15 May 2023  03:55:20 +0000 (0:00:00.156)       0:01:07.839 ************ 

TASK [download : prep_kubeadm_images | Generate list of required images] *******
ok: [node-0]
Monday 15 May 2023  03:55:20 +0000 (0:00:00.264)       0:01:08.103 ************ 

TASK [download : prep_kubeadm_images | Parse list of images] *******************
ok: [node-0] => (item=k8s.gcr.io/kube-apiserver:v1.20.7)
ok: [node-0] => (item=k8s.gcr.io/kube-controller-manager:v1.20.7)
ok: [node-0] => (item=k8s.g

<Result cmd='source myenv/bin/activate; cd kubespray; ansible-playbook -i inventory/mycluster/hosts.yaml  --become --become-user=root cluster.yml' exited=0>

In [ ]:
# give cc user the kube config file
remote.run("sudo cp -R /root/.kube /home/cc/.kube; sudo chown -R cc /home/cc/.kube; sudo chgrp -R cc /home/cc/.kube")

In [ ]:
# enable external access to the port on which we will deploy the service
for i in nodes.keys():
    remote = chi.ssh.Remote(nodes[i])
    remote.run("sudo ufw allow 32000")

In [ ]:
# also make sure docker can run without sudo 
# start a new SSH session after changing group membership
remote = chi.ssh.Remote(nodes[0])
remote.run("sudo groupadd docker; sudo usermod -aG docker $USER")
remote = chi.ssh.Remote(nodes[0])

In [ ]:
# copy docker configuration for private registry
remote.run("docker run -d -p 5000:5000 --restart always --name registry registry:2")
remote.run("sudo wget https://raw.githubusercontent.com/teaching-on-testbeds/k8s-ml/main/config/daemon.json -O /etc/docker/daemon.json")
remote.run("sudo service docker restart")

#### Validate your setup

Use the following command to validate your Kubernetes cluster with 3 servers:

In [ ]:
remote.run("kubectl get nodes")

Also test the Docker install:

In [ ]:
remote.run("docker run hello-world")

# Delete resources

### Delete server

In [ ]:
if True: 
    for i in servers:
      chi.server.delete_server(i.id)

#### De-configure Network



In [ ]:
router_id = router['id']
subnet_id = subnet['id']

try:
    result = chi.network.remove_subnet_from_router(router_id, subnet_id)
except Exception as e:
    print(f"detach_router_by_name error: {str(e)}")
    pass

try:
    result = chi.network.delete_router(router_id)
except Exception as e:
    print(f"delete_router_by_name error: {str(e)}")
    pass

try:
    result = chi.network.delete_subnet(subnet_id)
except Exception as e:
    print(f"delete_subnet_by_name error: {str(e)}")
    pass

try:
    result = chi.network.delete_network(network_id)
except Exception as e:
    print(f"delete_network_by_name error: {str(e)}")
    pass


#### Release Lease

In [ ]:
chi.lease.delete_lease(lease_id)